In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.utils.data_preprocessing import preprocess


In [4]:
books = pd.read_csv('../data/Books.csv', delimiter=';')
ratings = pd.read_csv('../data/Ratings.csv', delimiter=';')
users = pd.read_csv('../data/Users.csv', delimiter=';')

C:\Users\Никита\AppData\Local\Temp\ipykernel_3752\1781307168.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../data/Users.csv', delimiter=';')


In [8]:
df_prep_step_1 = pd.merge(books, ratings.query("`Rating` > 0"), on='ISBN', how='inner')

# join users data
df_prep_step_2 = pd.merge(df_prep_step_1, users, on='User-ID', how='inner')

In [10]:
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)
df_isbn = df_prep.drop_duplicates()

In [39]:
df_isbn

,ISBN,Title,Author,Year,User-ID,Rating,Age
0,0002005018,Clara Callan,Richard Bruce Wright,2001,8,5,NaN
1,0002005018,Clara Callan,Richard Bruce Wright,2001,11676,8,NaN
2,0002005018,Clara Callan,Richard Bruce Wright,2001,67544,8,30
3,0002005018,Clara Callan,Richard Bruce Wright,2001,116866,9,NaN
4,0002005018,Clara Callan,Richard Bruce Wright,2001,123629,9,NaN
...,...,...,...,...,...,...,...
363529,0679861548,Wolverine: Duty and Honor (X-Men Marvel Comics),Francine Hughes,1994,259901,7,NaN
363530,0816705739,"Good Luck, Bad Luck (Happy Times Adventures)",Rita Schlachter,1986,259901,10,NaN
363531,0822502763,Antonyms: Hot and Cold and Other Words That Ar...,Joan Hanson,1972,259901,10,NaN
363532,5550534274,Peter Pan Read-Along Book and Cassette,Walt Disney Productions,1989,259901,10,NaN


In [40]:
df_isbn['Original_NaN'] = df_isbn['Age'].isna()

# Convert 'Age' to numeric, turning non-numeric values into NaN
df_isbn['Age'] = pd.to_numeric(df_isbn['Age'], errors='coerce')

# Drop rows where 'Age' is NaN and were not originally NaN
df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]

# Drop the 'Original_NaN' column as it's no longer needed
df_isbn.drop(columns=['Original_NaN'], inplace=True)

C:\Users\Никита\AppData\Local\Temp\ipykernel_3752\701042267.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]
C:\Users\Никита\AppData\Local\Temp\ipykernel_3752\701042267.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isbn.drop(columns=['Original_NaN'], inplace=True)


In [41]:
df_isbn['User-ID'] = pd.to_numeric(df_isbn['User-ID'], errors='coerce')

# Drop rows where 'Age' is NaN and were not originally NaN
df_isbn = df_isbn[~(df_isbn['User-ID'].isna())]

C:\Users\Никита\AppData\Local\Temp\ipykernel_3752\423264576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isbn['User-ID'] = pd.to_numeric(df_isbn['User-ID'], errors='coerce')


In [42]:
age_outliers = df_isbn.query("Age > 100 or Age < 6")

user_outliers = age_outliers["User-ID"].to_list()

In [43]:
# exclude age outliers
df_isbn = df_isbn[~df_isbn["User-ID"].isin(user_outliers)]

In [45]:
df_isbn

,ISBN,Title,Author,Year,User-ID,Rating,Age
0,0002005018,Clara Callan,Richard Bruce Wright,2001,8,5,NaN
1,0002005018,Clara Callan,Richard Bruce Wright,2001,11676,8,NaN
3,0002005018,Clara Callan,Richard Bruce Wright,2001,116866,9,NaN
4,0002005018,Clara Callan,Richard Bruce Wright,2001,123629,9,NaN
5,0002005018,Clara Callan,Richard Bruce Wright,2001,200273,8,NaN
...,...,...,...,...,...,...,...
363529,0679861548,Wolverine: Duty and Honor (X-Men Marvel Comics),Francine Hughes,1994,259901,7,NaN
363530,0816705739,"Good Luck, Bad Luck (Happy Times Adventures)",Rita Schlachter,1986,259901,10,NaN
363531,0822502763,Antonyms: Hot and Cold and Other Words That Ar...,Joan Hanson,1972,259901,10,NaN
363532,5550534274,Peter Pan Read-Along Book and Cassette,Walt Disney Productions,1989,259901,10,NaN


In [ ]:
#from sklearn.impute import KNNImputer
#imputer = KNNImputer(n_neighbors=5)
#df_isbn['Age'] = imputer.fit_transform(df_isbn[['Age']])

In [ ]:
from src.recommenders.content_recommender import ContentRecommender

recommender = ContentRecommender()
users, ratings, books = preprocess(books, ratings, users)
recommender.fit(books, users, ratings)
